In [1]:
#Imports and GCS Setup: This cell imports the libraries and sets up the path to your data in the GCS bucket.

import os
import pandas as pd
import numpy as np
import pickle
from deepface import DeepFace
from google.cloud import storage
import gcsfs
from tqdm import tqdm # A library to show progress bars

# --- CONFIGURATION ---
# Make sure to replace this with your actual bucket name
BUCKET_NAME = "computer-vision-security-sys-data-hj" 
# --- END CONFIGURATION ---

# Path to the raw faces data in your bucket
GCS_FACES_PATH = f"gs://{BUCKET_NAME}/raw/faces/"

# This allows pandas to read directly from GCS
fs = gcsfs.GCSFileSystem()

print("Setup complete. GCS path is set to:", GCS_FACES_PATH)

Setup complete. GCS path is set to: gs://computer-vision-security-sys-data-hj/raw/faces/


In [2]:
# Discover Employee Data: This cell will list all the employee sub-folders you created in your GCS bucket.

# List all the directories (one for each employee) in the GCS path
employee_names = [os.path.basename(f) for f in fs.ls(GCS_FACES_PATH) if fs.isdir(f)]

if not employee_names:
    print("❌ Error: No employee folders found in your GCS bucket.")
    print("Please ensure your face data has been uploaded correctly.")
else:
    print("✅ Found employee folders:", employee_names)

✅ Found employee folders: ['himanshu_jangra', 'shivani_shahi']


In [3]:
# Generate, Average, and Store Embeddings
import tempfile

# The model we'll use for generating embeddings
# VGG-Face and Facenet are both excellent choices.
# MODEL_NAME = "DeepFace"
MODEL_NAME = "ArcFace"
# MODEL_NAME = "VGG-Face"
# MODEL_NAME = "FaceNet512"

employee_embeddings = {}

# Use tqdm for a nice progress bar
for name in tqdm(employee_names, desc="Processing Employees"):
    employee_folder_path = os.path.join(GCS_FACES_PATH, name)
    image_files = fs.ls(employee_folder_path)
    
    current_employee_embeddings = []
    
    for image_path in tqdm(image_files, desc=f"  -> {name}", leave=False):
        # The full GCS path for the image
        full_image_path = f"gs://{image_path}"

        # 1. Create a temporary file but tell it not to delete on close
        tmp_file = tempfile.NamedTemporaryFile(suffix=".jpg", delete=False)
        local_path = tmp_file.name
        
        # 2. Immediately close the file to release the Windows lock
        tmp_file.close()

        try:
            # Download the GCS file to the temporary local path
            fs.get(full_image_path, local_path)
            
            # DeepFace's represent function does the magic:
            # 1. Detects the face in the image.
            # 2. Generates the vector embedding.
            embedding_obj = DeepFace.represent(
                img_path=local_path,
                model_name=MODEL_NAME,
                enforce_detection=True # Ensures we don't process images without a face
            )
            
            # We only need the embedding vector itself
            current_employee_embeddings.append(embedding_obj[0]["embedding"])

        except ValueError as e:
            # This error is often thrown if a face isn't detected in an image
            print(f"\nWarning: Could not process {os.path.basename(image_path)}. Reason: {e}")
        except Exception as e:
            print(f"\nAn unexpected error occurred with {os.path.basename(image_path)}: {e}")

    if current_employee_embeddings:
        # Average the embeddings for all photos of one person for a more robust master embedding
        master_embedding = np.mean(current_employee_embeddings, axis=0)
        employee_embeddings[name] = master_embedding
        print(f"\n✅ Generated master embedding for {name}")
    else:
        print(f"\n❌ Failed to generate any embeddings for {name}. Please check the images.")

print("\n--- All employees processed. ---")

  -> himanshu_jangra:   4%|██████▉                                                                                                                                                                                 | 19/500 [00:13<03:30,  2.28it/s]


  -> himanshu_jangra:   5%|████████▍                                                                                                                                                                               | 23/500 [00:15<03:14,  2.45it/s]


  -> himanshu_jangra:   5%|████████▊                                                                                                                                                                               | 24/500 [00:16<03:10,  2.50it/s]


  -> himanshu_jangra:   5%|█████████▉                                                                                                                                                                              | 27/500 [00:17<02:54,  2.72it/s]


  -> himanshu_jangra:   6%|██████████▎                                                                                                                                                                             | 28/500 [00:17<02:43,  2.89it/s]

...


  -> shivani_shahi:  93%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍            | 466/500 [03:12<00:12,  2.78it/s]


  -> shivani_shahi:  95%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍         | 474/500 [03:15<00:09,  2.87it/s]


  -> shivani_shahi:  99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌ | 496/500 [03:24<00:01,  2.45it/s]


Processing Employees: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [06:52<00:00, 206.23s/it]


✅ Generated master embedding for shivani_shahi

--- All employees processed. ---


In [4]:
# Save the Face Database

# The location where your main processor script will look for the embeddings
SAVE_PATH = "../src/processor/face_embeddings.pkl"

with open(SAVE_PATH, "wb") as f:
    pickle.dump(employee_embeddings, f)

print(f"✅ Face database saved successfully to: {SAVE_PATH}")

✅ Face database saved successfully to: ../src/processor/face_embeddings.pkl
